In [ ]:
using Revise
using FUSE
using Plots;
FUSE.logging(Logging.Info; actors=Logging.Info);


In [ ]:
# sty is the act equivalent for a study, it has common parameters like server and n_workers but also study dependent parameters like n_simulations
sty,act = FUSE.study_parameters(:DatabaseGenerator);
sty

In [ ]:
# Interacting with sty
sty.server = "localhost"
sty.n_workers = 2

sty.save_folder = <YOUR SAVE FOLDER>

sty.release_workers_after_run = true # this is the default behavior and releases workers after running the study

sty.n_simulations = 4

sty

In [ ]:
# setup ini
ini, act = FUSE.case_parameters(:ITER;init_from=:scalars)

# core profiles
ini.core_profiles.zeff = 2.5 ↔ [1.4,4.5]
ini.core_profiles.greenwald_fraction_ped = 0.2 ↔[0.2,0.75*0.9]

# H&CD
ini.nb_unit[1].power_launched = 33.4e6 ↔ [0.0,33.4e6]
ini.nb_unit[1].beam_energy = 1e6 ↔ [1e5,1e6]
ini.ec_launcher[1].power_launched = 20e6 ↔ [0.0,20e6]
ini.ec_launcher[1].rho_0 = 0.3 ↔ [0.1,0.9]
ini.ic_antenna[1].power_launched = 24e6 ↔ [0.0,24e6]
ini.lh_antenna[1].power_launched = 10e6 ↔ [0.0,10e6]

# Change act for running ITER for IDA
act.ActorCoreTransport.model = :FluxMatcher
act.ActorFluxMatcher.evolve_pedestal = false
act.ActorTGLF.warn_nn_train_bounds = false
act.ActorFluxMatcher.evolve_rotation = :fixed
act.ActorFluxMatcher.evolve_densities = :flux_match

In [ ]:
# study is the actor equivalent of actors, here sty, act and outputs are kept of the workflow
study = FUSE.StudyDatabaseGenerator(sty,ini, act; n_workers=2); # it is possible to pass in keyword arguments to sty

# instantiating the study will also setup the study i.e. FUSE.setup(study);

In [ ]:
# for now you will have to import FUSE everywhere in your distributed computing
using Distributed
@everywhere import FUSE


In [ ]:
FUSE.run(study);   # runs the study, this also writes the dataframes

# at the end of the run workers are released so you will have to setup again if you want to run more

In [ ]:
# data for this workflow is stored in a dictionary of dataframes study.dataframes_dict
study.dataframes_dict["outputs_summary"]



In [ ]:
# analyze the study results 
FUSE.analyze(study);


